<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/08_LanguageModels/NgramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a entrenar un modelo de lenguaje ngram sobre un corpus de recetas de cocina con la librería `nltk`.

## Configuración del entorno

In [1]:
!pip install -qU datasets spacy nltk watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
%%capture
!python -m spacy download es_core_news_sm

In [3]:
%reload_ext watermark

In [4]:
%watermark -vmp datasets,spacy,nltk,numpy,pandas,tqdm

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

datasets: 3.0.0
spacy   : 3.7.6
nltk    : 3.9.1
numpy   : 1.26.4
pandas  : 2.1.4
tqdm    : 4.66.5

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.85+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



## Dataset

Vamos a usar un [corpus de recetas de SomosNLP](https://huggingface.co/datasets/somosnlp/RecetasDeLaAbuela).

In [5]:
from datasets import load_dataset

dataset = load_dataset("somosnlp/RecetasDeLaAbuela", "version_1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

main.csv:   0%|          | 0.00/40.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20236 [00:00<?, ? examples/s]

In [6]:
# vemos la estructura:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Id', 'Nombre', 'URL', 'Ingredientes', 'Pasos', 'Pais', 'Duracion', 'Categoria', 'Contexto', 'Valoracion y Votos', 'Comensales', 'Tiempo', 'Dificultad', 'Valor nutricional'],
        num_rows: 20236
    })
})


In [7]:
# Conservamos pais = "ESP":
dataset = dataset.filter(lambda x: x["Pais"] == "ESP")

Filter:   0%|          | 0/20236 [00:00<?, ? examples/s]

In [8]:
# vemos un ejemplo al azar:
dataset["train"][300]

{'Id': 1701,
 'Nombre': 'croquetas de pollo asado',
 'URL': 'https://www.recetasgratis.net/receta-de-croquetas-de-pollo-asado-59330.html',
 'Ingredientes': "['500 gramos de Pollo', '1 unidad de Cebolla', '300 mililitros de Aceite de oliva', '600 mililitros de Leche                                                                    (2½ tazas)', '2 cucharadas soperas de Harina', '1 pizca de Sal', '1 pizca de Nuez moscada molida', '1 pizca de Pimienta negra molida', '1 unidad de Huevo', '200 gramos de Pan rallado']",
 'Pasos': "['1 Reunimos todos los ingredientes para preparar las croquetas de pollo asado.', '2 Colocamos el pollo sin piel en una fuente para horno, salpimentamos, echamos un poco de aceite de oliva y lo metemos al horno a 180 ºC durante 25 minutos.También podéis hacer esta receta de croquetas de pollo asado si os ha sobrado carne de haber preparado, por ejemplo, pollo al horno con patatas y cebolla.', '3 Picamos la cebolla de forma fina y la salteamos en la sartén con un po

In [9]:
# A veces los textos son listas no parseadas como tales.
# En tal caso, hacemos un join de la lista.
import re

def preprocess(example):
    """
    """
    if example["Pasos"].startswith("["):
        pasos_list = eval(example["Pasos"].encode('unicode_escape'))
        example["Pasos"] = " ".join(pasos_list)
    # Eliminamos whitespace duplicado:
    example["Pasos"] = re.sub(r'\s+', ' ', example["Pasos"])
    return example

dataset = dataset.map(preprocess)

Map:   0%|          | 0/9381 [00:00<?, ? examples/s]

In [10]:
dataset["train"][300]

{'Id': 1701,
 'Nombre': 'croquetas de pollo asado',
 'URL': 'https://www.recetasgratis.net/receta-de-croquetas-de-pollo-asado-59330.html',
 'Ingredientes': "['500 gramos de Pollo', '1 unidad de Cebolla', '300 mililitros de Aceite de oliva', '600 mililitros de Leche                                                                    (2½ tazas)', '2 cucharadas soperas de Harina', '1 pizca de Sal', '1 pizca de Nuez moscada molida', '1 pizca de Pimienta negra molida', '1 unidad de Huevo', '200 gramos de Pan rallado']",
 'Pasos': '1 Reunimos todos los ingredientes para preparar las croquetas de pollo asado. 2 Colocamos el pollo sin piel en una fuente para horno, salpimentamos, echamos un poco de aceite de oliva y lo metemos al horno a 180 ºC durante 25 minutos.También podéis hacer esta receta de croquetas de pollo asado si os ha sobrado carne de haber preparado, por ejemplo, pollo al horno con patatas y cebolla. 3 Picamos la cebolla de forma fina y la salteamos en la sartén con un poco de ac

Hacemos un partición train/test y achicamos (solo para trabajar mas rapido). Y conservamos solo el texto de las recetas.

In [11]:
dataset = dataset.shuffle(seed=33)

In [12]:
texts_train = dataset["train"].select(range(0, 4_000))["Pasos"]
texts_test = dataset["train"].select(range(4_000, 8_000))["Pasos"]

In [13]:
import textwrap

print(textwrap.fill(texts_train[33], 100))

1 Tritura las galletas hasta hacerlas casi polvo, puedes ayudarte de una licuadora o una
procesadora. Si no dispones de estos utensilios, utiliza una bolsa plástica y un rodillo. 2 Derrite
la mantequilla hasta que quede líquida y agrégala a las galletas trituradas, mezcla bien hasta
formar una pasta uniforme. 3 Traslada la mezcla al molde y extiéndela para hacer la base de la tarta
de queso, puedes ayudarte de una paleta o hacerlo directamente con los dedos. Déjala reposar en el
refrigerador para que se endurezca mientras preparas el relleno. 4 Lleva la nata a fuego lento y
añade, poco a poco, el queso crema y el azúcar, siempre revolviendo con una paleta o batidor. 5 Para
añadir la cuajada, lo mejor es seguir las instrucciones de uso en su empaque. En este caso, la
disolvimos en 25 ml de leche y la agregamos a la mezcla mientras revolvíamos para una mejor
incorporación. 6 Pasados unos 5 minutos luego de añadir la cuajada, retira la mezcla del fuego y
deja reposar unos 2-3 minutos. Sac

## Tokenización y "entrenamiento" del LM

* Usamos el tokenizer para español de `spacy`.
* Consideramos como parte del vocabulario todas las palabras que ocurran al menos dos veces en train. Usamos `nltk` para definir el vocab y detectar los "\<unk\>" en test.
* Hacemos padding con BOS y EOS tokens.
* Debemos tener en cuenta que `nltk` espera que cada documento sea una lista de strings.

Usamos el LM más sencillo, el MLE (Maximum Likelihood Estimator), con 4-gramas.

In [14]:
# tokenizer con reglas de puntacion, contracciones, etc:
import spacy

tokenizer = spacy.load('es_core_news_sm')

In [15]:
# Veamos un ejemplo:
doc = tokenizer(texts_train[0])
print(doc.text)
for i, token in enumerate(doc):
    print(token.text)
    if i > 15:
        break

1 Cocemos el arroz en agua hirviendo con un poco de sal. 2 Picamos los ajos y la cebolla en cuadraditos.Cortamos el tocino en dados. 3 Cortamos las cabezas de la trucha por detrás de las agallas y con un cuchillo afilado, las abrimos de arriba abajo por el vientre. 4 Retiramos las vísceras y todas las membranas negras. 5 Seccionamos la carne a cada lado de la espina sin llegar al dorso. 6 Retiramos las espinas cortando a la altura de la cola. 7 Las lavamos muy bien y secamos con la ayuda de un paño. 8 Salpimentamos por dentro y por fuera.Lavamos las setas y las picamos. 9 Reservamos la mitad. 10 En una sartén con aceite, sofreímos el tocino y la mitad de las setas. 11 Rellenamos las truchas con las setas y el tocino. 12 Cerramos la abertura. 13 Horneamos durante 25 minutos. 14 Rehogamos en mantequilla el resto de las setas. 15 Añadimos el perejil picado, incorporamos el arroz cocido. 16 Vertemos un chorrito de aceite, añadimos el ajo picado, agregamos la cebolla picada y rehogamos dos 

In [16]:
# tokenizamos en train y test, sin marcar los UNK todavía:
import tqdm

def tokenize(doc, ngram_order=4):
    """Tokeniza un documento y agrega BOS y EOS tokens.
    """
    tokens = [token.text for token in tokenizer(doc)]
    tokens = ["<bos>"] * (ngram_order - 1) + tokens + ["<eos>"]
    return tokens

tokenized_train = []
for doc in tqdm.tqdm(texts_train):
    tokenized_train.append(tokenize(doc))

tokenized_test = []
for doc in tqdm.tqdm(texts_test):
    tokenized_test.append(tokenize(doc))

100%|██████████| 4000/4000 [03:01<00:00, 21.99it/s]


In [17]:
# Un poco de hacking de nltk para evitar que haga padding con la misma cantidad de tokens
# a izq y derecha
from functools import partial
from itertools import chain

from nltk.util import everygrams, pad_sequence

flatten = chain.from_iterable
pad_both_ends = partial(
    pad_sequence,
    pad_left=True,
    left_pad_symbol="<s>",
    pad_right=True,
    right_pad_symbol="</s>",
)

def padded_everygram_pipeline(order, text):
    """Modificación de https://www.nltk.org/_modules/nltk/lm/preprocessing.html
    para que no haga padding
    """
    padding_fn = partial(pad_both_ends, n=0)
    return (
        (everygrams(list(padding_fn(sent)), max_len=order) for sent in text),
        flatten(map(padding_fn, text)),
    )

train, train_flat = padded_everygram_pipeline(4, tokenized_train)

In [18]:
# Ahora sí armamos el vocab
from nltk.lm import Vocabulary

# cutoff de freq>=2 para el vocab:
vocab = Vocabulary(train_flat, unk_cutoff=2)

In [19]:
# vocab size:
len(vocab)

11341

In [20]:
# los tokens más y menos frecuentes:
print(sorted(vocab.counts, key=vocab.counts.get, reverse=True)[:5])
print(sorted(vocab.counts, key=vocab.counts.get)[:5])

['.', 'de', ',', 'y', 'la']
['vientre', 'Seccionamos', 'prendemos', 'digestivo', 'bridar']


In [21]:
# los tokens ordenados alfabeticamente:
print(sorted(vocab.counts)[:5])
print(sorted(vocab.counts, reverse=True)[:5])

['!', '"', '"LISTO', '%', '&']
['💪', '🍲', '€', '…', '”']


In [22]:
# los tokens con frec 1 "no están en el vocab" (pero podemos consultar su frec.)
print(vocab["el"], "el" in vocab)
print(vocab["digestivo"], "digestivo" in vocab)
print(vocab[" "], " " in vocab)
print(vocab["riquelme"], "riquelme" in vocab)

25861 True
1 False
0 False
0 False


In [23]:
# ejemplos de sequencia tokenizada:
print(vocab.lookup(tokenized_train[33][:10]))
print(vocab.lookup(["un", "té", "digestivo", "."]))

('<bos>', '<bos>', '<bos>', '1', 'Tritura', 'las', 'galletas', 'hasta', 'hacerlas', 'casi')
('un', 'té', '<UNK>', '.')


In [24]:
# instanciamos el modelo con el ngram order y el vocab
from nltk.lm import MLE

lm = MLE(4, vocabulary=vocab)

In [25]:
%%time
lm.fit(train)

CPU times: user 34.3 s, sys: 738 ms, total: 35 s
Wall time: 35.6 s


In [26]:
print(lm.counts)

<NgramCounter with 4 ngram orders and 3938852 ngrams>


In [27]:
# unigram counts
lm.counts['la']

33884

In [28]:
# bigram counts
print(lm.counts[['en']]["la"])
print(lm.counts[['la']]["en"])
print(lm.counts[['el']]["<UNK>"])

2418
0
204


In [29]:
# trigram y 4gram counts
print(lm.counts[["con", "la"]]["cuchara"])
print(lm.counts[["y", "con", "la"]]["cuchara"])

14
2


In [30]:
# lo mas frecuente despues de un ngrama dado:
ngram_example = ["con", "la"]
sorted(lm.counts[ngram_example].items(), key=lambda x: x[1], reverse=True)[:10]

[('salsa', 239),
 ('receta', 193),
 ('ayuda', 141),
 ('preparación', 136),
 ('cebolla', 103),
 ('leche', 100),
 ('masa', 96),
 ('mezcla', 96),
 ('mantequilla', 95),
 ('crema', 74)]

In [31]:
# probabilidad de un token luego de un ngrama:
ngram_example = ["con", "la", "salsa"]
print(lm.score("rosa", ngram_example))

0.0041841004184100415


In [32]:
# usamos logscore para evitar underflow:
import numpy as np

ngram_example = ["con", "la", "salsa"]
print(lm.logscore("rosa", ngram_example))
print(np.log2(lm.score("rosa", ngram_example)))

-7.90086680798075
-7.900866807980749


## Evaluación

Medimos perplexity en el dataset de test.

In [33]:
example_test = tokenized_test[33]
print(example_test)
print(lm.vocab.lookup(example_test))

['<bos>', '<bos>', '<bos>', '1', 'Para', 'empezar', 'con', 'la', 'receta', 'del', 'budín', 'de', 'naranja', 'fácil', ',', 'primero', 'bate', 'el', 'huevo', 'a', 'temperatura', 'ambiente', 'con', 'la', 'pizca', 'de', 'sal', 'mientras', 'agregas', 'el', 'azúcar', 'en', 'forma', 'de', 'lluvia', '.', '2', 'Cuando', 'el', 'azúcar', 'esté', 'integrado', 'y', 'no', 'se', 'observen', 'los', 'cristales', 'del', 'mismo', ',', 'añade', 'el', 'aceite', 'en', 'forma', 'de', 'hilo', 'mientras', 'continúas', 'batiendo', '.', '3', 'Cuando', 'el', 'aceite', 'esté', 'integrado', 'y', 'el', 'batido', 'esté', 'espumoso', 'y', 'blanquecino', ',', 'agrega', 'la', 'ralladura', 'de', 'la', 'naranja', '.', 'Por', 'otro', 'lado', ',', 'exprime', 'la', 'naranja', '.', 'Debes', 'saber', 'que', 'necesitarás', 'media', 'taza', 'de', 'líquido', ',', 'por', 'tanto', ',', 'si', 'no', 'alcanzas', 'a', 'llenarla', 'con', 'el', 'jugo', 'de', 'naranja', ',', 'puedes', 'completarla', 'con', 'agua', ',', 'leche', 'o', 'yogu

In [34]:
from nltk.util import ngrams

def perplexity(docs, lm, ngram_order=3) -> float:
    """docs: lista de listas de tokens (con BOS y EOS)
    """
    ngrams_flat = []
    for doc in docs:
        ngrams_ = ngrams(doc, ngram_order)
        ngrams_flat.extend(list(ngrams_))
    return lm.perplexity(ngrams_flat)

In [35]:
%%time
ppl_train = perplexity(tokenized_train, lm)
print(f"Perplexity en train: {ppl_train:.4f}")

Perplexity en train: 8.1328
CPU times: user 10.8 s, sys: 288 ms, total: 11.1 s
Wall time: 11.2 s


In [36]:
%%time
ppl_test = perplexity(tokenized_test, lm)
print(f"Perplexity en test: {ppl_test:.4f}")
# qué pasó??

Perplexity en test: inf
CPU times: user 11.6 s, sys: 126 ms, total: 11.7 s
Wall time: 11.8 s


Necesitamos smoothing / backoff / interpolation para computar perplexity en test!

Usamos add-k smoothing (aka Lidstone smoothing, gamma=k)

In [37]:
from nltk.lm import Lidstone

train, train_flat = padded_everygram_pipeline(4, tokenized_train)
vocab = Vocabulary(train_flat, unk_cutoff=2)
lm_smoothed = Lidstone(order=4, vocabulary=vocab, gamma=.01)

In [38]:
%%time
lm_smoothed.fit(train)

CPU times: user 33.3 s, sys: 554 ms, total: 33.8 s
Wall time: 34.1 s


In [39]:
ppl_test_smoothed = perplexity(tokenized_test, lm_smoothed)
print(f"Perplexity en test: {ppl_test_smoothed:.4f}")

Perplexity en test: 199.4909


## Generación de texto

Generamos texto sampleando iterativamente del LM.

In [40]:
tokens_generados = lm_smoothed.generate(
    30, text_seed=["<bos>", "<bos>", "<bos>"], random_seed=33)
print(" ".join(tokens_generados))

1 Para preparar esta receta que te haga el trabajo de marcar los medios como <UNK> <UNK> , el <UNK> natural por excelencia . 5 Por último , agrega unas


In [43]:
import textwrap

tokens_generados = lm_smoothed.generate(
    120, text_seed=["<bos>", "<bos>", "1"], random_seed=33)
receta_generada = " ".join(tokens_generados)
print(textwrap.fill(receta_generada, 100))

Para hacer las capas de relleno pues deberán estar frías una vez vayan a ser utilizadas . Prepara en
primer lugar limpiaremos los calamares , que , bien escurridas y aliñamos con un poco de arroz
blanco . 2 Añade el chocolate en baño María con una olla a fuego lento 2 minutos más . 4 Este es el
primer paso es alistar todos los ingredientes para elaborar los calabacines rellenos de gambas y
parmesano . Truco : Aunque recomendamos hacer tres capas . 7 Finalmente servir las acelgas al puré y
procesar nuevamente , hasta obtener una mezcla homogénea y suave . Si quieres recetas más detalles
para aprender a preparar más croquetas , no te pierdas esta sencilla
